# Intro to JSON and Calling LLM APIs in Python


### What you'll learn
- JSON format basics (objects, arrays, values) and how it maps to Python types
- Python's `json` module: `load`, `loads`, `dump`, `dumps`
- How LLM chat APIs structure messages (`system`, `user`, `assistant`)
- How to call an LLM via OpenRouter (using the free **MiniMax M2** model)
- Two mini-projects: sentiment analysis (binary) and emotion classification (multi-class)



## Part 1 — JSON 101

**JSON** (JavaScript Object Notation) is a lightweight text format to exchange data.
- **Object** = unordered set of key–value pairs, written with `{}`.
- **Array** = ordered list of values, written with `[]`.
- Values can be **string**, **number**, **boolean**, **null**, **object**, **array**.

**Example JSON:**

```json
{
  "title": "Spirited Away",
  "year": 2001,
  "genres": ["Animation", "Fantasy"],
  "rating": 8.6,
  "is_family_friendly": true,
  "extra": null
}
```


### Using Python's `json` module

In [1]:
import json

# JSON string -> Python (loads)
json_str = '{"title": "Spirited Away", "year": 2001, "genres": ["Animation", "Fantasy"], "rating": 8.6}'
data = json.loads(json_str)
print(type(data), data)
print("Title:", data["title"])

# Python -> JSON string (dumps)
py_obj = {"ok": True, "items": [1, 2, 3], "note": "hello"}
json_pretty = json.dumps(py_obj, indent=2, ensure_ascii=False)
print(json_pretty)

# Working with files: dump / load
with open("example.json", "w", encoding="utf-8") as f:
    json.dump(py_obj, f, indent=2, ensure_ascii=False)

with open("example.json", "r", encoding="utf-8") as f:
    loaded = json.load(f)
print("Loaded back:", loaded)


<class 'dict'> {'title': 'Spirited Away', 'year': 2001, 'genres': ['Animation', 'Fantasy'], 'rating': 8.6}
Title: Spirited Away
{
  "ok": true,
  "items": [
    1,
    2,
    3
  ],
  "note": "hello"
}
Loaded back: {'ok': True, 'items': [1, 2, 3], 'note': 'hello'}



## Part 2 — How LLM chat APIs work

Most chat APIs use a **message list**. Each message has a **role** and **content**:

```json
[
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "Explain bubble sort in one sentence."},
  {"role": "assistant", "content": "Bubble sort repeatedly swaps adjacent items..."}
]
```

- **system**: high-level instructions (tone, constraints, persona).
- **user**: the user's request or question.
- **assistant**: the model's reply.



### The model we'll use: MiniMax M2 (free on OpenRouter)

**Links**
- MiniMax M2 (free) on OpenRouter: https://openrouter.ai/minimax/minimax-m2%3Afree
- All free models on OpenRouter: https://openrouter.ai/models/?q=free
- MiniMax docs (OpenAI-compatible API): https://platform.minimax.io/docs/api-reference/text-openai-api
- OpenRouter on reasoning tokens & `reasoning_details`: https://openrouter.ai/docs/use-cases/reasoning-tokens

**Description (from the model page):**

> MiniMax-M2 is a compact, high-efficiency large language model optimized for end-to-end coding and agentic workflows. With 10 billion activated parameters (230 billion total), it delivers near-frontier intelligence across general reasoning, tool use, and multi-step task execution while maintaining low latency and deployment efficiency.
>
> The model excels in code generation, multi-file editing, compile-run-fix loops, and test-validated repair, showing strong results on SWE-Bench Verified, Multi-SWE-Bench, and Terminal-Bench. It also performs competitively in agentic evaluations such as BrowseComp and GAIA, effectively handling long-horizon planning, retrieval, and recovery from execution errors.
>
> Benchmarked by Artificial Analysis, MiniMax-M2 ranks among the top open-source models for composite intelligence, spanning mathematics, science, and instruction-following. Its small activation footprint enables fast inference, high concurrency, and improved unit economics, making it well-suited for large-scale agents, developer assistants, and reasoning-driven applications that require responsiveness and cost efficiency.
>
> To avoid degrading this model's performance, MiniMax highly recommends preserving reasoning between turns. Learn more about using reasoning_details to pass back reasoning in our docs.


### Calling the API via OpenRouter (OpenAI-compatible client)

In [12]:
OPENROUTER_API_KEY = ""

In [3]:
# !pip install --upgrade openai pandas  # uncomment if needed
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

# Minimal call (user-only)
completion = client.chat.completions.create(
    model="minimax/minimax-m2:free",
    messages=[{"role": "user", "content": "What is the meaning of life?"}],
)
print(completion.choices[0].message.content)


That's the ultimate question—and the most beautiful thing about it is that **there is no single, universal answer**. The meaning of life has been explored by philosophy, religion, science, and art for millennia, and each perspective offers unique insights. Here are some common lenses through which people understand "meaning":

### 1. **Meaning through Connection**
   - **Relationships**: Love, friendship, and community give life purpose. Helping others, raising children, or building bonds creates deep fulfillment.
   - **Legacy**: Many find meaning in contributing to something larger than themselves—art, science, social change, or leaving the world better for future generations.

### 2. **Meaning through Purpose**
   - **Personal Passion**: Following your interests, pursuing creativity, or excelling in your craft can make life feel significant.
   - **Service**: Devoting oneself to causes like justice, compassion, or environmental stewardship can anchor life with profound purpose.

###

#### More examples

In [4]:
# Example 1: Add a system prompt
resp = client.chat.completions.create(
    model="minimax/minimax-m2:free",
    messages=[
        {"role": "system", "content": "You are a kind CS tutor for beginners. Keep answers short."},
        {"role": "user", "content": "Explain bubble sort in simple words."}
    ],
)
print(resp.choices[0].message.content)

# Example 2: Force a small JSON answer
resp = client.chat.completions.create(
    model="minimax/minimax-m2:free",
    messages=[
        {"role": "system", "content": "Reply ONLY with a JSON object and nothing else."},
        {"role": "user", "content": "Extract the movie title and year from: 'I saw Spirited Away (2001) last night.'"}
    ],
)
print(resp.choices[0].message.content)

# Example 3: Quick translation
resp = client.chat.completions.create(
    model="minimax/minimax-m2:free",
    messages=[
        {"role": "system", "content": "Translate to Polish. No extra commentary."},
        {"role": "user", "content": "JSON is a simple format for data exchange."}
    ],
)
print(resp.choices[0].message.content)


### Bubble sort in simple words

- It’s like walking through a line and “bubble up” the big numbers by swapping neighbors that are out of order.
- Step-by-step:
  1. Go from left to right, compare each pair of neighbors.
  2. If the left one is bigger than the right, swap them.
  3. After one pass, the largest value reaches the far right.
  4. Repeat the pass on the whole list (except the already placed last item).
  5. Stop when you finish a full pass with no swaps—everything is sorted.
- Example: [4, 2, 3, 1]
  - Pass 1: compare/swap pairs → [2, 3, 1, 4] (largest “bubbled” to the end)
  - Pass 2: → [2, 1, 3, 4]
  - Pass 3: → [1, 2, 3, 4] (no swaps, done)

- It’s simple and stable (keeps equal items in their original order), but slow for large lists (worst/average O(n²)).
{
  "title": "Spirited Away",
  "year": 2001
}
JSON to prosty format do wymiany danych.


## Part 3 — Project: Sentiment analysis (IMDB data)

**Goal:** Read the first 100 reviews, keep 10 for a tiny demo, and classify each as `positive`, `negative`, or `neutral` using the LLM.
If parsing fails, write `"error"` in the column. We'll iterate row-by-row and show progress with `tqdm`.

In [5]:
import pandas as pd
from openai import OpenAI

# ---- Data prep ----
csv_path = "imdb_100.csv"
df_all = pd.read_csv(csv_path, index_col=0)
df = df_all.head(10).copy()  # keep only first 10 rows for demo
df["sentiment"] = ""  # add empty column with sentiment

df

,review,sentiment
0,One of the other reviewers has mentioned that ...,
1,A wonderful little production. <br /><br />The...,
2,I thought this was a wonderful way to spend ti...,
3,Basically there's a family where a little boy ...,
4,"Petter Mattei's ""Love in the Time of Money"" is...",
5,"Probably my all-time favorite movie, a story o...",
6,I sure would like to see a resurrection of a u...,
7,"This show was an amazing, fresh & innovative i...",
8,Encouraged by the positive comments about this...,
9,If you like original gut wrenching laughter yo...,


In [1]:
# ---- LLM setup ----
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=OPENROUTER_API_KEY)

def classify_sentiment(review: str) -> str:
    try:
        print()
        print()
        print("Classifying review:", review)
        prompt = (
            'Classify movie review sentiment as one of: positive, negative, neutral.\n'
            'Return ONLY a JSON object like {"sentiment": "positive"}. No extra text and without ``` marks.'
        )
        resp = client.chat.completions.create(
            model="minimax/minimax-m2:free",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": review},
            ],
        )
        response_content = resp.choices[0].message.content.strip()  # get model output - it's a string
        print("Model response:", response_content)
        obj = json.loads(response_content)  # try to parse as JSON
        print("Parsed JSON:", obj)
        return obj['sentiment']
    except Exception as e:
        print("Could not parse review:", e)
        return "error"

NameError: name 'OpenAI' is not defined

Let's try it out!

In [7]:
result = classify_sentiment("That movie was surprisingly good!")
print("Classified sentiment:", result)



Classifying review: That movie was surprisingly good!
Model response: {"sentiment":"positive"}
Parsed JSON: {'sentiment': 'positive'}
Classified sentiment: positive


In [8]:
result = classify_sentiment("That movie was mid.")
print("Classified sentiment:", result)



Classifying review: That movie was mid.
Model response: {"sentiment": "neutral"}
Parsed JSON: {'sentiment': 'neutral'}
Classified sentiment: neutral


Finally, let's classify all our reviews from the dataframe!

In [9]:
for i in range(len(df)):
    df.loc[i, "sentiment"] = classify_sentiment(df.loc[i, "review"])

df



Classifying review: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive



## Part 4 — Multi-class classification (emotions)

Now let's try more categories: `joy`, `sadness`, `anger`, `fear`, `surprise`, `neutral`.
Same approach: one row at a time, expect **pure JSON** back, otherwise mark as `"error"`.


In [10]:
import json

df_emotions = df.copy()
df_emotions["emotion"] = ""

def classify_emotion(review: str) -> str:
    try:
        print()
        print()
        print("Classifying review:", review)
        prompt = (
            'Classify the emotion in this movie review as one of: joy, sadness, anger, fear, surprise, neutral.'
            'Return ONLY a JSON object like {"emotion": "joy"}. No extra text and without ``` marks.'
        )
        resp = client.chat.completions.create(
            model="minimax/minimax-m2:free",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": review},
            ],
        )
        response_content = resp.choices[0].message.content.strip()  # get model output - it's a string
        print("Model response:", response_content)
        obj = json.loads(response_content)  # try to parse as JSON
        print("Parsed JSON:", obj)
        if obj['emotion'] in {"joy", "sadness", "anger", "fear", "surprise", "neutral"}:
            return obj['emotion']
        return "???"
    except Exception as e:
        print("Could not parse review:", e)
        return "error"

In [11]:
for i in range(len(df_emotions)):
    df_emotions.loc[i, "emotion"] = classify_emotion(df_emotions.loc[i, "review"])

df_emotions



Classifying review: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the 

,review,sentiment,emotion
0,One of the other reviewers has mentioned that ...,positive,joy
1,A wonderful little production. <br /><br />The...,positive,joy
2,I thought this was a wonderful way to spend ti...,positive,joy
3,Basically there's a family where a little boy ...,negative,anger
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,neutral
5,"Probably my all-time favorite movie, a story o...",positive,joy
6,I sure would like to see a resurrection of a u...,positive,joy
7,"This show was an amazing, fresh & innovative i...",negative,anger
8,Encouraged by the positive comments about this...,negative,anger
9,If you like original gut wrenching laughter yo...,positive,joy
